<a href="https://colab.research.google.com/github/ZaFuh/deepsort/blob/main/Sahi_Deepsort_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!unzip Trial5.zip


Archive:  Trial5.zip
   creating: Trial5/
  inflating: Trial5/best.pt          
   creating: Trial5/deep_sort/
   creating: Trial5/deep_sort/.git/
   creating: Trial5/deep_sort/.git/branches/
  inflating: Trial5/deep_sort/.git/config  
  inflating: Trial5/deep_sort/.git/description  
  inflating: Trial5/deep_sort/.git/HEAD  
   creating: Trial5/deep_sort/.git/hooks/
  inflating: Trial5/deep_sort/.git/hooks/applypatch-msg.sample  
  inflating: Trial5/deep_sort/.git/hooks/commit-msg.sample  
  inflating: Trial5/deep_sort/.git/hooks/fsmonitor-watchman.sample  
  inflating: Trial5/deep_sort/.git/hooks/post-update.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-applypatch.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-commit.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-merge-commit.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-push.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-rebase.sample  
  inflating: Trial5/deep_sort/.git/hooks/pre-receive.sample  

In [4]:
cd Trial5


/content/Trial5


In [5]:
ls

best.pt     main.py      requirements.txt  vid1.mp4      vid3.mp4.mp4
deep_sort/  model_data/  tracker.py        vid2.mp4.mp4


In [6]:
%pip install ultralytics

In [7]:
!pip -q install ultralytics sahi==0.11.15 lap

In [17]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [20]:
!sudo apt-get update
!sudo apt-get install -y build-essential cmake
!pip install -r requirements.txt

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [23]:
# Read the original requirements.txt
with open('requirements.txt', 'r') as f:
    lines = f.readlines()

# Filter out the scikit-learn line with the specific version
new_lines = [line for line in lines if not line.strip().startswith('scikit-learn==')]

# Add scikit-learn without a version specifier (or you could add a compatible version like 'scikit-learn>=1.2.0')
new_lines.append('scikit-learn\n')

# Write the modified content back to requirements.txt
with open('requirements.txt', 'w') as f:
    f.writelines(new_lines)

print("Updated requirements.txt")

Updated requirements.txt


In [24]:
pip install -r requirements.txt

  Using cached ultralytics-8.0.44-py3-none-any.whl.metadata (24 kB)
ERROR: Ignored the following versions that require a different python version: 8.0.10 Requires-Python >=3.7,<=3.11; 8.0.11 Requires-Python >=3.7,<=3.11; 8.0.12 Requires-Python >=3.7,<=3.11; 8.0.13 Requires-Python >=3.7,<=3.11; 8.0.14 Requires-Python >=3.7,<=3.11; 8.0.15 Requires-Python >=3.7,<=3.11; 8.0.16 Requires-Python >=3.7,<=3.11; 8.0.17 Requires-Python >=3.7,<=3.11; 8.0.18 Requires-Python >=3.7,<=3.11; 8.0.19 Requires-Python >=3.7,<=3.11; 8.0.20 Requires-Python >=3.7,<=3.11; 8.0.21 Requires-Python >=3.7,<=3.11; 8.0.22 Requires-Python >=3.7,<=3.11; 8.0.23 Requires-Python >=3.7,<=3.11; 8.0.24 Requires-Python >=3.7,<=3.11; 8.0.25 Requires-Python >=3.7,<=3.11; 8.0.26 Requires-Python >=3.7,<=3.11; 8.0.27 Requires-Python >=3.7,<=3.11; 8.0.28 Requires-Python >=3.7,<=3.11; 8.0.29 Requires-Python >=3.7,<=3.11; 8.0.30 Requires-Python >=3.7,<=3.11; 8.0.31 Requires-Python >=3.7,<=3.11; 8.0.32 Requires-Python >=3.7,<=3.11; 8.

In [25]:
# Read the original requirements.txt
with open('requirements.txt', 'r') as f:
    lines = f.readlines()

# Filter out the ultralytics and tensorflow lines with specific versions
new_lines = [line for line in lines if not line.strip().startswith('ultralytics==') and not line.strip().startswith('tensorflow==')]

# Add ultralytics and tensorflow without version specifiers
new_lines.append('ultralytics\n')
new_lines.append('tensorflow\n')


# Write the modified content back to requirements.txt
with open('requirements.txt', 'w') as f:
    f.writelines(new_lines)

print("Updated requirements.txt to remove version constraints for ultralytics and tensorflow.")

Updated requirements.txt to remove version constraints for ultralytics and tensorflow.


In [26]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 105.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [8]:
#YASSS THIS IS THE ONE Read the original requirements.txt
with open('requirements.txt', 'r') as f:
    lines = f.readlines()

# Filter out the scikit-image line with the specific version
new_lines = [line for line in lines if not line.strip().startswith('scikit-image==')]

# Add scikit-image without a version specifier
new_lines.append('scikit-image\n')

# Write the modified content back to requirements.txt
with open('requirements.txt', 'w') as f:
    f.writelines(new_lines)

print("Updated requirements.txt to remove version constraint for scikit-image.")

Updated requirements.txt to remove version constraint for scikit-image.


In [9]:
pip install -r requirements.txt

In [11]:
#along with all previous steps from 4.py, install SAHI: !pip -q install ultralytics sahi==0.11.15 lap

import os
import cv2
import torch
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel

# allowlist DetectionModel for torch weights-only unpickler (safe if you trust weights)
torch.serialization.add_safe_globals([DetectionModel])

# DeepSORT imports
from deep_sort.deep_sort.tracker import Tracker as DeepSort
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection

# SAHI imports
from sahi.models.yolov8 import Yolov8DetectionModel
from sahi.predict import get_sliced_prediction

def xyxy_to_xywh(box):
    """Convert [x1,y1,x2,y2] box to [cx,cy,w,h]."""
    x1, y1, x2, y2 = box
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2
    cy = y1 + h / 2
    return [cx, cy, w, h]

# Paths
VIDEO_IN = "vid1.mp4"
VIDEO_OUT = "out_tracked_sahi.mp4"
WEIGHTS = "best.pt"  # your YOLOv11 weights

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# (Optional) Load YOLO just to access class names if you want later
ultra_model = YOLO(WEIGHTS)
ultra_model.to(device)
names = ultra_model.model.names if hasattr(ultra_model, "model") else {}

# SAHI detection model (works with Ultralytics v8/v11 under the hood)
sahi_model = Yolov8DetectionModel(
    model_path=WEIGHTS,
    confidence_threshold=0.25,  # tune as needed
    device=device,
)

# Init DeepSORT
max_cosine_distance = 0.4
nn_budget = None
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = DeepSort(metric)

# Open video
cap = cv2.VideoCapture(VIDEO_IN)
if not cap.isOpened():
    raise RuntimeError(f"Could not open input video {VIDEO_IN}")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

# Safer mp4 codec
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))

# SAHI slice settings (good starting point for 1080p+; tune for your footage)
SLICE_W = 640
SLICE_H = 640
OVERLAP_W = 0.2
OVERLAP_H = 0.2
# If objects are *really* tiny, increase overlap (e.g., 0.3–0.4) or reduce slice size to 512

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # ---- SAHI sliced inference on this frame ----
    # Note: pass the frame (BGR numpy array) directly
    sahi_result = get_sliced_prediction(
        image=frame,
        detection_model=sahi_model,
        slice_height=SLICE_H,
        slice_width=SLICE_W,
        overlap_height_ratio=OVERLAP_H,
        overlap_width_ratio=OVERLAP_W,
        perform_standard_pred=False,         # only sliced pred (faster when you only want slices)
        postprocess_match_metric="IOU",      # merge overlapping slice preds
        postprocess_match_threshold=0.5,     # tune if you see duplicate boxes
    )

    # Convert SAHI predictions -> DeepSORT detections
    detections = []
    for obj in sahi_result.object_prediction_list:
        # bbox in xyxy
        x1, y1, x2, y2 = obj.bbox.to_xyxy()
        conf = float(obj.score.value)
        # SAHI category.id may be None depending on model metadata; default to 0 if missing
        cls_id = int(obj.category.id) if obj.category and obj.category.id is not None else 0

        bbox_xywh = xyxy_to_xywh([x1, y1, x2, y2])
        detections.append(Detection(bbox_xywh, conf, cls_id))

    # Update tracker with detections
    tracks = tracker.update(detections)

    # Draw tracks (ID only; add class name if you want)
    if tracks is not None:
        for track in tracks:
            bbox = track.to_tlbr()
            x1, y1, x2, y2 = map(int, bbox)
            track_id = int(track.track_id)

            # Optional: show class label (requires mapping track->last detection class if you store it)
            # Here we just draw the ID for simplicity
            color = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID:{track_id}", (x1, y1 - 6),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Ensure frame size matches writer and save
    frame_resized = cv2.resize(frame, (w, h))
    writer.write(frame_resized)

cap.release()
writer.release()
print("✅ Saved video:", VIDEO_OUT)


Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing prediction on 2 number of slices.
Performing

In [10]:
!pip install "numpy<2" --upgrade
